## Setup spark and install modules

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import regexp_extract

In [2]:
spark = SparkSession.builder.appName("LogsAnalysis").getOrCreate()

your 131072x1 screen size is bogus. expect trouble
24/04/08 08:56:28 WARN Utils: Your hostname, DELEQ0283302041 resolves to a loopback address: 127.0.1.1; using 172.31.227.62 instead (on interface eth0)
24/04/08 08:56:28 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/08 08:56:29 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/04/08 08:56:30 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


## Setup


### Generate data
For out practice purposes, we will generate fake Apache Access logs.
>1. Open terminal and change directory to "Data" folder
>2. In terminal, run command `python log_generator apache_access_schema.yaml`, add flag `-t` to truncate previously created logs
>3. Based on provide .yaml file, new `log.txt.{id}` file should appear every 5s with 50 lines of fake logs.

## Practice

1. Create DataFrame representing the stream of raw log data lines ariving to `/'log/apache_access` directory

In [3]:
access_lines = spark.readStream.text("../../mock_logs/log/apache_access")

2. Parse log data to DataFrame

In [4]:
# Parse out the common log format to a DataFrame
contentSizeExp = r'\s(\d+)$'
statusExp = r'\s(\d{3})\s'
generalExp = r'\"(\S+)\s(\S+)\s*(\S*)\"'
timeExp = r'\[(\d{2}/\w{3}/\d{4}:\d{2}:\d{2}:\d{2}:\d{4})]'
hostExp = r'(^\S+\.[\S+\.]+\S+)\s'

logsDF = access_lines.select(regexp_extract('value', hostExp, 1).alias('host'),
                        regexp_extract('value', timeExp, 1).alias('timestamp'),
                        regexp_extract('value', generalExp, 1).alias('method'),
                        regexp_extract('value', generalExp, 2).alias('endpoint'),
                        regexp_extract('value', generalExp, 3).alias('protocol'),
                        regexp_extract('value', statusExp, 1).cast('integer').alias('status'),
                        regexp_extract('value', contentSizeExp, 1).cast('integer').alias('content_size'))

3. Count every access status code

In [5]:
statusCountsDF=logsDF.groupBy("status").count()

4. Kick off our streaming query, dumping results to the console

In [6]:
query = statusCountsDF.writeStream.outputMode("complete").format("console").queryName("status_counts").start()
query.awaitTermination()

-------------------------------------------
Batch: 0
-------------------------------------------
+------+-----+
|method|count|
+------+-----+
| PATCH|    9|
|  POST|    7|
|DELETE|   15|
|   PUT|    9|
|   GET|   10|
+------+-----+



-------------------------------------------
Batch: 1
-------------------------------------------
+------+-----+
|method|count|
+------+-----+
| PATCH|   14|
|  POST|    8|
|DELETE|   15|
|   PUT|   12|
|   GET|   11|
+------+-----+



-------------------------------------------
Batch: 2
-------------------------------------------
+------+-----+
|method|count|
+------+-----+
| PATCH|   15|
|  POST|   10|
|DELETE|   16|
|   PUT|   13|
|   GET|   11|
+------+-----+



-------------------------------------------
Batch: 3
-------------------------------------------
+------+-----+
|method|count|
+------+-----+
| PATCH|   20|
|  POST|   17|
|DELETE|   23|
|   PUT|   17|
|   GET|   13|
+------+-----+



-------------------------------------------
Batch: 4
-------------------------------------------
+------+-----+
|method|count|
+------+-----+
| PATCH|   23|
|  POST|   18|
|DELETE|   24|
|   PUT|   17|
|   GET|   13|
+------+-----+



...

-------------------------------------------
Batch: 6
-------------------------------------------
+------+-----+
|method|count|
+------+-----+
| PATCH|   25|
|  POST|   21|
|DELETE|   27|
|   PUT|   19|
|   GET|   16|
+------+-----+



-------------------------------------------
Batch: 7
-------------------------------------------
+------+-----+
|method|count|
+------+-----+
| PATCH|   25|
|  POST|   27|
|DELETE|   28|
|   PUT|   22|
|   GET|   20|
+------+-----+



-------------------------------------------
Batch: 8
-------------------------------------------
+------+-----+
|method|count|
+------+-----+
| PATCH|   28|
|  POST|   36|
|DELETE|   31|
|   PUT|   26|
|   GET|   26|
+------+-----+

-------------------------------------------
Batch: 9
-------------------------------------------
+------+-----+
|method|count|
+------+-----+
| PATCH|   31|
|  POST|   36|
|DELETE|   31|
|   PUT|   27|
|   GET|   28|
+------+-----+



ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/home/mnestoro/SPARK/.venv/lib/python3.10/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/home/mnestoro/SPARK/.venv/lib/python3.10/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

5. Classroom cleanup
IMPORTANT: Kill running log_generator script in terminal

In [11]:
query.stop()
spark.stop()